In [1]:
import numpy as np
from scipy.integrate import quad

In [2]:
# Constants (SI units)

q = 1.602176634e-19                 # Elementary charge (C)
m_star = 0.32 * 9.10938356e-31      # Effective mass (kg)
hbar = 1.0545718e-34                # Reduced Planck's constant (J.s)
eps_0 = 8.854187817e-12  # Vacuum permittivity (F/m)
eps_s = 8.9  # Static dielectric constant
eps_inf = 5.35  # High-frequency dielectric constant


n_2D = 1e17  # 2D carrier density (m^-2)

k_F = np.sqrt(2 * np.pi * n_2D)                               # Fermi wavevector (1/m)
q_TF = (m_star * q**2)/(2*np.pi *eps_s*eps_0 * hbar**2)    # Thomas-Fermi wavevector (1/m)


L = 20e-10  #(m)
delta = 3e-10  #(m)


In [3]:
def G(q, b):
    t = b / (b + q)
    return (1/8)*(2*t**3 + 3*t**2 + 3*t)


b = np.cbrt(33*m_star* (q**2) *n_2D/(8* (hbar**2) *eps_0*eps_s))


In [4]:

def calculate_tau_ifr(m_star, q, delta, L, n_2D, eps_0, eps_s, k_F, q_TF, G):
    """
    Calculate the inverse of the relaxation time due to interface roughness scattering (1/tau_IFR).

    Parameters:
        m_star (float): Effective mass of the electron.
        q (float): Electron charge (C).
        delta_ln2D (float): Roughness parameter.
        eps_0 (float): Vacuum permittivity (F/m).
        eps_s (float): Static dielectric constant.
        L (float): Characteristic length of roughness (m).
        k_F (float): Fermi wave vector (1/m).
        q_TF (float): Thomas-Fermi screening wave vector (1/m).
        G (callable): Screening form factor function G(u).
        hbar (float): Reduced Planck's constant (J.s).

    Returns:
        float: Inverse of relaxation time due to interface roughness scattering (1/tau_IFR).
    """
    # Prefactor
    prefactor = (m_star * (q**2 * delta* L* n_2D)**2) / (8 * (eps_0 * eps_s)**2 * hbar**3)

    # Define the integrand function
    def integrand(u):
        numerator = u**4 * np.exp(-(L * k_F * u)**2)
        denominator = (u + q_TF * G(u, b) / (2 * k_F))**2 * np.sqrt(1 - u**2)
        return numerator / denominator

    # Perform the integration from 0 to 1
    integral, _ = quad(integrand, 0, 1)

    # Calculate 1/tau_IFR
    tau_ifr_inv = prefactor * integral

    return tau_ifr_inv

# Calculate 1/tau_IFR
tau_ifr_inv = calculate_tau_ifr(m_star, q, delta, L, n_2D, eps_0, eps_s, k_F, q_TF, G)

print(f"Inverse relaxation time (1/tau_IFR): {tau_ifr_inv:.2e} s^-1")
mobility = (q) / (tau_ifr_inv * m_star)
print("Mobility:", mobility*1e4, "cm^2/Vs")



Inverse relaxation time (1/tau_IFR): 3.60e+11 s^-1
Mobility: 15273.431498645847 cm^2/Vs


In [5]:
import numpy as np
from scipy.integrate import quad

def mobility_ifr(delta, L, n_2D, m_star, eps_0, eps_s, q):
    """
    Calculate the mobility limited by interface roughness scattering (IFR).

    Parameters:
        delta (float): Roughness amplitude (m).
        L (float): Roughness correlation length (m).
        n_2D (float): 2D carrier density (m^-2).
        m_star (float): Effective mass of the electron (kg).
        eps_0 (float): Vacuum permittivity (F/m).
        eps_s (float): Static dielectric constant.
        q (float): Elementary charge (C).

    Returns:
        float: Mobility limited by interface roughness scattering (cm^2/V·s).
    """
    # Constants
    hbar = 1.0545718e-34  # Reduced Planck's constant (J·s)

    # Derived parameters
    k_F = np.sqrt(2 * np.pi * n_2D)  # Fermi wavevector (1/m)
    q_TF = (m_star * q**2) / (2 * np.pi * eps_s * eps_0 * hbar**2)  # Thomas-Fermi wavevector (1/m)

    # Screening form factor function G(u)
    def G(q, b):
        t = b / (b + q)
        return (1/8) * (2 * t**3 + 3 * t**2 + 3 * t)

    # Screening parameter b
    b = np.cbrt(33 * m_star * (q**2) * n_2D / (8 * (hbar**2) * eps_0 * eps_s))

    # Define the integrand for the relaxation time
    def integrand(u):
        numerator = u**4 * np.exp(-(L * k_F * u)**2)
        denominator = (u + q_TF * G(u, b) / (2 * k_F))**2 * np.sqrt(1 - u**2)
        return numerator / denominator

    # Perform the integration from 0 to 1
    integral, _ = quad(integrand, 0, 1)

    # Prefactor for 1/tau_IFR
    prefactor = (m_star * (q**2 * delta * L * n_2D)**2) / (8 * (eps_0 * eps_s)**2 * hbar**3)
    tau_ifr_inv = prefactor * integral  # Inverse of relaxation time (1/tau_IFR)

    # Calculate mobility
    mobility_ifr = (q) / (tau_ifr_inv * m_star)  # Mobility in m^2/V·s
    return mobility_ifr * 1e4  # Convert to cm^2/V·s


# Example Usage
delta = 3e-10  # Roughness amplitude (m)
L = 20e-10  # Roughness correlation length (m)
n_2D = 1e17  # 2D carrier density (m^-2)
m_star = 0.32 * 9.10938356e-31  # Effective mass (kg)
eps_0 = 8.854187817e-12  # Vacuum permittivity (F/m)
eps_s = 8.9  # Static dielectric constant
q = 1.602176634e-19  # Elementary charge (C)

# Calculate mobility
mobility = mobility_ifr(delta, L, n_2D, m_star, eps_0, eps_s, q)
print(f"Mobility due to interface roughness scattering: {mobility:.2f} cm^2/V·s")


Mobility due to interface roughness scattering: 15273.43 cm^2/V·s
